## Homework Assigmment 1.6 p.11 from Lecture notes

An airline seeks a reservation policy for a flight with $S$ seats that maximizes its expected profit from the flight. Reservation requests arrive hourly according to a Bernoulli process with $p$ being the probability of a reservation request per hour (at most one reservation request will arrive per hour). A passenger with a booked reservation pays the fare $f > 0$ at flight time. If $b \geq 0$ passengers with booked reservations are denied boarding at flight time, they do not pay the fare, and the airline pays them a penalty $c(b)$ (divided among them) where $b \mapsto c(b)$ is increasing with $c(0) = 0$.

Consider the $n$-th hour before flight time $T$. At the beginning of the hour, the airline reviews the number of booked reservations on hand, $r$ say, and decides whether to book (accept) or decline a reservation request arriving during the next hour. Each of the $r$ booked reservations may cancel during the hour, independently of each other, with probability $q$.

For this reason, the airline is considering the possibility of overbooking the flight to compensate for cancellations. Let $V^*_n(r)$ be the maximum expected future profit when there are $r$ booked reservations at the beginning of the hour, before the accept/decline decision has been taken, and reservation requests and cancellations during the hour have occurred. Let $W^*_n(r)$ be the maximum expected future profit when there are $r$ booked reservations after booking or declining a reservation request, but before cancellations. The aim is to determine an optimal reservation policy for any value of the number of booked reservations at the beginning of each hour till the flight time $T$.

### a) Markov Decision Model

Formulate the problem as a Markov decision model, by determining the state space, action spaces, rewards, terminal rewards, and the transition probabilities. Formulate the optimality equation from which an optimal reservation policy can be determined.

### b) Optimality of Booking-Limit Policies

Assume, as can be shown, that if $g$ is a quasiconcave function on the integers, then $r \mapsto \mathbb{E}(g(B_r))$ is quasiconcave, where $B_r$ is a sum of independent identically distributed Bernoulli random variables. We recall that $g$ is quasiconcave on the (positive) integers when there exists a number $a$ such that $g$ is increasing on $[0, a]$ and decreasing on $[a, \infty]$.

Use this result to show the following facts. First, show that $r \mapsto W^*_n(r)$ is quasiconcave.

Let $b_n = \arg\max_r W^*_n(r)$. Call $b_n$ the booking limit. Then show that $r \mapsto V^*_n(r)$ is quasiconcave with maximum $b_n$. Finally, show that it is optimal to accept a reservation if and only if $r < b_n$, with $r$ the number of booked reservations on hand at the beginning of the hour (before a decision has been taken).

### c) Solving the Problem

Solve the problem when the parameters are as follows:

-   $T = 30$
-   $c(b) = f \cdot b$
-   $S = 10$
-   $f =$ € $300$
-   $p = 0.2$ and $0.3$
-   $q = 0.05$ and $0.10$
-   $r \leq 20$ (so there is an upper bound on the total number of reservations).

Make graphs of the different combinations. In each case, estimate the booking limit ten hours before flight time from your graphs. Discuss whether your graphs confirm the claim in (b) that $r \mapsto V^*_n(r)$ is quasiconcave.

What conjectures do the graphs suggest about the optimal reservation policy and/or maximum expected reward and their variation with the various data elements? You will lose points on your conjectures only if your graphs are inconsistent with or do not support your conjectures, or if you don’t make enough interesting conjectures. The idea here is to brainstorm intelligently.

## Solutions

### c) Solving the Problem

::: panel-tabset
## Code


In [1]:
# Load packages
from scipy.stats import binom
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Functions
def Value_calc(n, r, c, p, V_opt, prob_c):
    value = prob_c*(p*V_opt[n-1][r+1-c] + (1-p)*V_opt[n-1][r-c])
    return value

def solve_mdp(T, f, S, p, q, r_max, penalty=lambda d, b: 0):

    V_opt = {}
    policy_opt = {}

    for n in range(T+1):

        V_opt[n] = {}
        policy_opt[n] = {}

        if n==0:
            for r in  range(T+1):
                if r <= S:
                    V_opt[n][r] = f*r
                else:
                    V_opt[n][r] = f*S - penalty(f, r - S)

        else:
            for r in range(T+1 - n):
            
                cancellations = {
                    c: binom.pmf(k=c, n=r, p=q) for c in range(r)
                }
                V_accept = sum([
                    Value_calc(n, r, c, p, V_opt, prob_c)
                    for c, prob_c in cancellations.items()
                ])
                V_reject = sum([
                    Value_calc(n, r, c, 0, V_opt, prob_c)
                    for c, prob_c in cancellations.items()
                ])

                if r >= r_max:
                    policy_opt[n][r] = "reject"
                    V_opt[n][r] = V_reject
                else:
                    if V_accept >= V_reject:
                        policy_opt[n][r] = "accept"
                        V_opt[n][r] = V_accept
                    else:
                        policy_opt[n][r] = "reject"
                        V_opt[n][r] = V_reject
    
    return policy_opt, V_opt

def print_policy(policy_opt, V_opt):
    for n, policy_n in policy_opt.items():
        print(f"Epochs before flight: {n}")
        for r in policy_n:
            print(f"When number of bookings is {r} then policy: {policy_n[r]}, with value {V_opt[n][r]:.2f}")

def plot_policy(policy_opt, V_opt, n, title):
    data_dict = {'r': policy_opt[n].keys(), 'policy': policy_opt[n].values(), 'V_star': V_opt[n].values()}
    data = pd.DataFrame(data_dict)
    fig = px.line(data, x = 'r', y = 'V_star', color='policy', title = title, markers=True)
    fig.show()

def plot_policies(T, f, S, r_max, probs=[(0.2, 0.05), (0.2, 0.1), (0.3, 0.05), (0.3, 0.1)], n=10, file_name="airline-policies.png"):
    fig = go.Figure()
    
    # Get a list of colors to use for different (p, q) pairs
    color_sequence = px.colors.qualitative.Plotly  # Default Plotly color sequence
    probs_colors = {}  # Dictionary to map each (p, q) to a specific color

    for idx, (p, q) in enumerate(probs):
        policy_opt, V_opt = solve_mdp(T, f, S, p, q, r_max, penalty=penalty_func)
        data_dict = {
            'r': list(policy_opt[10].keys()),
            'policy': list(policy_opt[10].values()),
            'V_star': list(V_opt[10].values())
        }
        data = pd.DataFrame(data_dict)
        data = data.sort_values('r')

        # Assign a consistent color to this (p, q) pair
        color = color_sequence[idx % len(color_sequence)]
        probs_colors[(p, q)] = color

        # Identify segments where policy is the same
        segments = []
        current_policy = None
        current_segment = {'r': [], 'V_star': [], 'policy': None}
        for index, row in data.iterrows():
            policy = row['policy']
            if current_policy is None or policy != current_policy:
                if current_segment['r']:
                    segments.append(current_segment)
                current_policy = policy
                current_segment = {'r': [], 'V_star': [], 'policy': policy}
            current_segment['r'].append(row['r'])
            current_segment['V_star'].append(row['V_star'])
        segments.append(current_segment)  # Append the last segment

        # Plot each segment with appropriate line style and consistent color
        segment_count = 0
        for segment in segments:
            line_dash = 'solid' if segment['policy'] == 'accept' else 'dot'
            showlegend = segment_count == 0  # Show legend only for the first segment
            segment_count += 1
            fig.add_trace(go.Scatter(
                x=segment['r'],
                y=segment['V_star'],
                mode='lines+markers',
                name=f'p={p}, q={q}',
                text = segment['policy'],
                legendgroup=f'p={p}, q={q}',
                showlegend=showlegend,
                line=dict(dash=line_dash, color=color)
            ))

    # Edit the layout
    
    fig.update_layout(
        title=f"Optimal values and policies {n} epochs before flight",
        xaxis_title='Reservations (r)',
        yaxis_title='Optimal value (V*)',
    )
    fig.write_image(file_name, width=800, height=600, scale=3) # Save the plot as an image                                   
    fig.show()

## Documentation

#### Function Documentation: `solve_mdp`

**Overview**

The `solve_mdp` function solves a finite-horizon Markov Decision Process (MDP) to determine an optimal reservation policy for an airline. The goal is to maximize the airline's expected profit from a flight with a limited number of seats by making hourly decisions on whether to accept or reject incoming reservation requests. The function accounts for overbooking due to potential cancellations and applies penalties for denied boarding.

**Function Signature**

``` python
def solve_mdp(T, f, S, p, q, r_max, penalty=lambda d, b: 0):
```

*Parameters*

-   `T` : int\
    The total number of hours before flight time. This is the time horizon over which decisions are made.

-   `f` : float\
    The fare paid by each passenger who successfully boards the flight. This fare represents the reward associated with accepting a reservation.

-   `S` : int\
    The number of available seats on the flight. This is the maximum number of passengers that can board.

-   `p` : float\
    The probability of receiving a reservation request in each hour. This is the arrival rate of reservation requests.

-   `q` : float\
    The probability that a booked reservation is canceled. Each reservation has this independent probability of cancellation.

-   `r_max` : int\
    The upper limit for the number of reservations the airline is willing to accept (overbooking threshold). If the number of reservations exceeds this value, new requests are automatically rejected.

-   `penalty` : function (optional)\
    A function that calculates the penalty for denied boarding based on the number of overbooked passengers. The default penalty function is `lambda d, b: 0`, which implies no penalty for denied boarding.

*Returns*

-   `policy_opt` : dict\
    A dictionary that provides the optimal action (`"accept"` or `"reject"`) at each time step `n` and number of reservations `r`. It has the structure `policy_opt[n][r]`.

-   `V_opt` : dict\
    A dictionary that provides the maximum expected future profit at each time step `n` and reservation count `r`. It has the structure `V_opt[n][r]`.

**Helper Function:** `Value_calc`

``` python
def Value_calc(n, r, c, p, V_opt, prob_c):
```

This helper function computes the expected value for accepting or rejecting a reservation, factoring in the probability of cancellations:

$$
\text{Value} = \text{prob}_c \cdot (p \cdot V_{\text{opt}}[n-1][r+1-c] + (1-p) \cdot V_{\text{opt}}[n-1][r-c])
$$

Where:

-   `n`: The current time step (hour before the flight).

-   `r`: The current number of reservations.

-   `c`: The number of cancellations.

-   `p`: The probability of a reservation request.

-   `V_opt`: The dictionary storing the value function.

-   `prob_c`: The probability of having `c` cancellations (based on the binomial distribution).

**Methodology**

The function uses dynamic programming with backward induction to compute the optimal reservation policy and the expected profit. The key challenge is balancing the potential profit from accepting more reservations against the risk of overbooking and paying penalties for denied boarding.

**Value Function Initialization**

At the final time step (`n = 0`), the value function is initialized based on the number of reservations `r`:

$$
V_0(r) = 
\begin{cases} 
f \cdot r & \text{if } r \leq S \\
f \cdot S - f \cdot (r - S) - c(r - S) & \text{if } r > S
\end{cases}
$$

This represents the profit earned from passengers who successfully board the flight. If the number of reservations exceeds the seating capacity (`r > S`), the fare for the overbooked passengers is forfeited, and the airline pays a penalty `c(b)`, where `b = r - S`.

**Recursive Value Calculation**

For each time step `n > 0`, the function calculates the value of accepting (`V_{\text{accept}}`) or rejecting (`V_{\text{reject}}`) a reservation request. This is done using the `Value_calc` function, which computes the expected profit, taking cancellations into account. The value of accepting a reservation is: $$
V_{\text{accept}} = \sum_{c=0}^{r} \binom{r}{c} q^c (1 - q)^{r - c} \left[ p \cdot V_{\text{opt}}[n-1][r+1-c] + (1-p) \cdot V_{\text{opt}}[n-1][r-c] \right]
$$ The value of rejecting a reservation is: $$
V_{\text{reject}} = \sum_{c=0}^{r} \binom{r}{c} q^c (1 - q)^{r - c} V_{\text{opt}}[n-1][r-c]
$$

**Overbooking and Penalties**

If the number of reservations exceeds the capacity (`r > S`), the function applies a penalty `c(b)` where `b = r - S` represents the number of overbooked passengers. The profit is adjusted to account for this penalty.

**Policy Decision**

At each time step and reservation level: - If `r >= r_max`, the policy is automatically set to `"reject"` (to prevent excessive overbooking). - Otherwise, the policy compares the expected value of accepting vs. rejecting the reservation and selects the action that maximizes the expected profit: $$
\text{policy}_{n}(r) = 
\begin{cases} 
\text{"accept"} & \text{if } V_{\text{accept}} \geq V_{\text{reject}} \\
\text{"reject"} & \text{otherwise}
\end{cases}
$$

**Example Usage**

``` python
T = 10    # Time steps (hours before flight)
f = 100   # Fare per passenger
S = 50    # Seating capacity
p = 0.8   # Probability of a reservation request
q = 0.1   # Cancellation probability
r_max = 60  # Maximum allowable reservations

policy, value = solve_mdp(T, f, S, p, q, r_max, penalty)
```

**Notes**

-   The function uses the binomial probability mass function (`binom.pmf`) to model the cancellation process. Each reservation has an independent probability `q` of being canceled.

-   The penalty for denied boarding, `c(b)`, can be adjusted based on the problem's requirements, where `b = r - S` is the number of overbooked passengers.

-   The function solves the MDP by backward induction, starting from the final time step and working backward to the first time step to compute the optimal policy.
:::

#### Case


In [3]:
T = 30
f = 300
S = 10
p_cases = [0.2, 0.3]
q_cases = [0.05, 0.1]
r_max = 20
penalty_func = lambda d, b: d * b

#### Results


In [4]:
probs = [(0.2, 0.05), (0.2, 0.1), (0.3, 0.05), (0.3, 0.1)]
n=10
plot_policies(T, f, S, r_max, probs, n)

This chart shows the optimal values and policies for different combinations of $p$ and $q$ ten hours before the flight. The solid lines represent the optimal policy of accepting reservations, while the dotted lines represent the optimal policy of rejecting reservations. The gap in the lines indicates the level of $r$ where the policy switches from 'accept' to 'reject' The color coding indicates different combinations of $p$ and $q$.

#### Conjectures

**Conjecture 1** - The booking limit $b_n$ depends only on $q$ and not on $p$ and increases with $q$.

Comparing the blue and green or red and purple lines ($p$ varies, $q$ is fixed) it is clear that the booking limit $b_n$ is independent of $p$. While comparing the blue and red lines or green and purple ($p$ is fixed, $q$ varies) it is clear that the booking limit $b_n$ increases with $q$.

**Conjecture 2** - The optimal value $V^*_n(r)$ decreases with $q$ and increases with $p$.

Comparing the blue and red or green and purple lines ($q$ varies, $p$ is fixed) it is clear that the optimal value $V^*_n(r)$ decreases with $q$. While comparing the blue and green or red and purple lines ($q$ is fixed, $p$ varies) it is clear that the optimal value $V^*_n(r)$ increases with $p$.

#### Cases


In [5]:
p = p_cases[0]
q = q_cases[0]
n = 10
title = f"Optimal values and policies with p={p}, q={q} and {n} epochs before flight"
policy_opt, V_opt = solve_mdp(T, f, S, p, q, r_max, penalty=penalty_func)
plot_policy(policy_opt, V_opt, n, title)
print_policy(policy_opt, V_opt)

Epochs before flight: 0
Epochs before flight: 1
When number of bookings is 0 then policy: accept, with value 0.00
When number of bookings is 1 then policy: accept, with value 342.00
When number of bookings is 2 then policy: accept, with value 629.85
When number of bookings is 3 then policy: accept, with value 914.99
When number of bookings is 4 then policy: accept, with value 1200.00
When number of bookings is 5 then policy: accept, with value 1485.00
When number of bookings is 6 then policy: accept, with value 1770.00
When number of bookings is 7 then policy: accept, with value 2055.00
When number of bookings is 8 then policy: accept, with value 2340.00
When number of bookings is 9 then policy: accept, with value 2625.00
When number of bookings is 10 then policy: reject, with value 2850.00
When number of bookings is 11 then policy: reject, with value 2793.72
When number of bookings is 12 then policy: reject, with value 2566.80
When number of bookings is 13 then policy: reject, with va

In [6]:
p = p_cases[0]
q = q_cases[1]
n = 10
title = f"Optimal values and policies with p={p}, q={q} and {n} epochs before flight"
policy_opt, V_opt = solve_mdp(T, f, S, p, q, r_max, penalty=penalty_func)
plot_policy(policy_opt, V_opt, n, title)
print_policy(policy_opt, V_opt)

Epochs before flight: 0
Epochs before flight: 1
When number of bookings is 0 then policy: accept, with value 0.00
When number of bookings is 1 then policy: accept, with value 324.00
When number of bookings is 2 then policy: accept, with value 599.40
When number of bookings is 3 then policy: accept, with value 869.94
When number of bookings is 4 then policy: accept, with value 1139.99
When number of bookings is 5 then policy: accept, with value 1410.00
When number of bookings is 6 then policy: accept, with value 1680.00
When number of bookings is 7 then policy: accept, with value 1950.00
When number of bookings is 8 then policy: accept, with value 2220.00
When number of bookings is 9 then policy: accept, with value 2490.00
When number of bookings is 10 then policy: accept, with value 2718.16
When number of bookings is 11 then policy: reject, with value 2781.71
When number of bookings is 12 then policy: reject, with value 2675.14
When number of bookings is 13 then policy: reject, with va

In [7]:
p = p_cases[1]
q = q_cases[0]
n = 10
title = f"Optimal values and policies with p={p}, q={q} and {n} epochs before flight"
policy_opt, V_opt = solve_mdp(T, f, S, p, q, r_max, penalty=penalty_func)
plot_policy(policy_opt, V_opt, n, title)
print_policy(policy_opt, V_opt)

Epochs before flight: 0
Epochs before flight: 1
When number of bookings is 0 then policy: accept, with value 0.00
When number of bookings is 1 then policy: accept, with value 370.50
When number of bookings is 2 then policy: accept, with value 659.78
When number of bookings is 3 then policy: accept, with value 944.99
When number of bookings is 4 then policy: accept, with value 1230.00
When number of bookings is 5 then policy: accept, with value 1515.00
When number of bookings is 6 then policy: accept, with value 1800.00
When number of bookings is 7 then policy: accept, with value 2085.00
When number of bookings is 8 then policy: accept, with value 2370.00
When number of bookings is 9 then policy: accept, with value 2655.00
When number of bookings is 10 then policy: reject, with value 2850.00
When number of bookings is 11 then policy: reject, with value 2793.72
When number of bookings is 12 then policy: reject, with value 2566.80
When number of bookings is 13 then policy: reject, with va

In [8]:
p = p_cases[1]
q = q_cases[1]
n = 10
title = f"Optimal values and policies with p={p}, q={q} and {n} epochs before flight"
policy_opt, V_opt = solve_mdp(T, f, S, p, q, r_max, penalty=penalty_func)
plot_policy(policy_opt, V_opt, n, title)
print_policy(policy_opt, V_opt)

Epochs before flight: 0
Epochs before flight: 1
When number of bookings is 0 then policy: accept, with value 0.00
When number of bookings is 1 then policy: accept, with value 351.00
When number of bookings is 2 then policy: accept, with value 629.10
When number of bookings is 3 then policy: accept, with value 899.91
When number of bookings is 4 then policy: accept, with value 1169.99
When number of bookings is 5 then policy: accept, with value 1440.00
When number of bookings is 6 then policy: accept, with value 1710.00
When number of bookings is 7 then policy: accept, with value 1980.00
When number of bookings is 8 then policy: accept, with value 2250.00
When number of bookings is 9 then policy: accept, with value 2520.00
When number of bookings is 10 then policy: accept, with value 2727.24
When number of bookings is 11 then policy: reject, with value 2781.71
When number of bookings is 12 then policy: reject, with value 2675.14
When number of bookings is 13 then policy: reject, with va